In [1]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', None)

import datetime as dt

In [2]:
today = dt.datetime.now()
year = today.year

In [3]:
# get file paths
curr_filepath = input('input file location of report 4-1: ') #../data_and_output/110817/R4_1_Report.xlsx
prev_filepath = input('input file location of previous report: ') #../data_and_output/4-2 direct_master_q3.xlsx

input file location of report 4-1: 1
input file location of previous report: 2


In [4]:
# read in data
df_curr = pd.read_excel(curr_filepath) # report 4-1
df_prev = pd.read_excel(prev_filepath) # previous report

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
df_curr['Batch'] = input('input batch number (eg. 2023 Q3): ')

input batch number (eg. 2023 Q3): Q4


In [6]:
df_curr[['Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception']] = ""

In [7]:
df_prev['Date of Birth']  = pd.to_datetime(df_prev['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date
df_prev['Date of Incorporation']  = pd.to_datetime(df_prev['Date of Incorporation'], format='%Y-%m-%d', errors='coerce').dt.date

df_prev['dob_year'] = df_prev['Date of Birth'].apply(lambda x: x.year)
df_prev['doi_year'] = df_prev['Date of Incorporation'].apply(lambda x: x.year)

In [8]:
df_curr['Date of Birth']  = pd.to_datetime(df_curr['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date
df_curr['Date of Incorporation']  = pd.to_datetime(df_curr['Date of Incorporation'], format='%Y-%m-%d', errors='coerce').dt.date

df_curr['dob_year'] = df_curr['Date of Birth'].apply(lambda x: x.year)
df_curr['doi_year'] = df_curr['Date of Incorporation'].apply(lambda x: x.year)

In [9]:
df_prev['key'] = df_prev['Portfolio Number'].astype('str') + df_prev['Party ID'].astype('str')
df_curr['key'] = df_curr['Portfolio Number'].astype('str') + df_curr['Party ID'].astype('str')

In [10]:
# filter active status
df_curr = df_curr[df_curr['Current Status'] == 'Active']

In [11]:
#filter dob/doi
df_curr = df_curr[(year-df_curr['dob_year']>100) | (year-df_curr['doi_year']>100)]
df_prev = df_prev[(year-df_prev['dob_year']>100) | (year-df_prev['doi_year']>100)]

In [12]:
# at this point we have finished preparing the current report
# now merge with the records in the previous report

In [13]:
# identify parties that appeared in previous report
prev_parties = df_prev['key'].tolist()

In [14]:
# identify parties that appear in current report
curr_parties = df_curr['key'].tolist()

In [15]:
# list of parties that appear in both previous and current report (unresolved or exceptions)
both_parties = set(prev_parties).intersection(set(curr_parties))

In [16]:
# list of new parties that appear only in the current report
new_parties = set(curr_parties).difference(set(prev_parties))

In [17]:
# total unresolved = old unresolved (both_parties) and new unresolved (new_parties)
df_curr = df_curr[(df_curr['key'].isin(both_parties)) | (df_curr['key'].isin(new_parties))]

In [18]:
# update old unresolved parties with details from previous remediation
cols = ['Batch', 'Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception']
for col in cols:
    df_curr.loc[df_curr['key'].isin(both_parties),col] = df_prev.loc[df_prev['key'].isin(both_parties),col].tolist()

In [19]:
# drop extra columns, rearrange columns
df_curr = df_curr.drop(columns=['dob_year','doi_year', 'key'])

df_curr = df_curr[['Batch', 'Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                   'Portfolio Number', 'Portfolio Name', 'Current Status', 'Relationship Type', 'Relationship Sub-Type', 
                   'RM Name', 'RM Location Name', 'RM Team Name', 'Booking Center Officer Name','Managed By EAM / EIA / Finder',
                   'EAM / EIA Name', 'Finder Name','Portfolio Opening Date', 'Relationship (Party Role)', 'Party Name',
                   'Party Type', 'Party ID', 'Is Public', 'Date of Birth','Date of Incorporation']]

df_curr = df_curr.sort_values(by='Batch')

In [20]:
# print to excel
df_curr.to_excel('direct_party_dob-doi.xlsx', index=False)